We got this python notebook from the following GitHub repo (https://github.com/subpath/Keras_music_gereration) as a way to test our baseline:

### Music gereration firth Keras and TensorFlow backend

Plan was simple:
1. Read midi file, convert it to matrix of features
2. Create simple model with Keras and LSTM to learn the pattern
3. Use subsample of initial midi file as a input for model to generate pure art
4. Save prediction from model to midi file
.
.
.
5. PROFIT

<i> For disclamer: I've been using my old Dell Laptop with no GPU support</i>

In [76]:
import mido
import os
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np

##### Read midi file

In [77]:
mid = MidiFile('Data/Bamboleo.mid')  
# midis = []
# print(os.listdir('Data'))
# for file in os.listdir('Data'):
#     if file.endswith(".mid"):
#         print(file)
#         midis.append(MidiFile('Data/' + file, clip=True))
    #files.append(file)

In [4]:
# mid = MidiFile('Data/abusowillieCol.mid', clip=True)

# for i, track in enumerate(mid.tracks):
#     print('Track {}: {}'.format(i, track.name))
#     for msg in track:
#         if msg.type == 'program_change':
#             print(msg)

##### Extract notes sequence

In [88]:
notes = []

def isPercussion(channel):
    return channel in range(8, 17) or channel in range(35, 82)

for msg in mid:
#         if msg.type == 'program_change':
#             print(msg)
    if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
        data = msg.bytes()
        notes.append(data[1])

# for file in os.listdir('Data')[1:6]:
#     mid = MidiFile('Data/' + file, clip=True)
#     for msg in mid:
# #         if msg.type == 'program_change':
# #             print(msg)
#         if not msg.is_meta and msg.channel == 0 and msg.type == 'note_on':
#             data = msg.bytes()
#             notes.append(data[1])

#### Apply min-max scalling

In [83]:
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(np.array(notes).reshape(-1,1))
notes = list(scaler.transform(np.array(notes).reshape(-1,1)))

/Users/StephanieBrito/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


#### Prepare features for training and data subsample for prediction

In [101]:
# LSTM layers requires that data must have a certain shape
# create list of lists fist
#notes = [list(note) for note in notes]

# subsample data for training and prediction
X = []
y = []
# number of notes in a batch
n_prev = 2
for i in range(len(notes)-n_prev):
    X.append(notes[i:i+n_prev])
    y.append(notes[i+n_prev])
# save a seed to do prediction later
X_test = X[-300:]
X = np.asarray(X[:-300])
y = y[:-300]
print(X[0])
print(len(set(y)))

[37 49]
24


In [102]:
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
print(len(dummy_y[0]))

24


#### Made sequential model with several layers, use LSTM as it time dependent data

I also whant to save checkpoints

In [106]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# define baseline model
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=2, activation='relu'))
    model.add(Dense(24, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=20, batch_size=5, verbose=1)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/20
3427/3427 [==============================] - 5s 1ms/step - loss: 11.0551 - acc: 0.0986
Epoch 2/20
3427/3427 [==============================] - 1s 376us/step - loss: 10.4918 - acc: 0.1240
Epoch 3/20
3427/3427 [==============================] - 1s 368us/step - loss: 10.4631 - acc: 0.1161
Epoch 4/20
3427/3427 [==============================] - 1s 363us/step - loss: 10.4528 - acc: 0.1153
Epoch 5/20
3427/3427 [==============================] - 1s 384us/step - loss: 10.4457 - acc: 0.12140s -
Epoch 6/20
3427/3427 [==============================] - 2s 448us/step - loss: 10.4435 - acc: 0.1179
Epoch 7/20
3427/3427 [==============================] - 1s 417us/step - loss: 10.4431 - acc: 0.1191
Epoch 8/20
3427/3427 [==============================] - 1s 372us/step - loss: 10.4370 - acc: 0.1170
Epoch 9/20
3427/3427 [==============================] - 1s 377us/step - loss: 10.4346 - acc: 0.1176
Epoch 10/20
3427/3427 [==============================] - 1s 385us/step - loss: 10.4321 - acc: 0.12

3427/3427 [==============================] - 1s 345us/step - loss: 13.7131 - acc: 0.1024
Epoch 20/20
381/381 [==============================] - 1s 3ms/step
Epoch 1/20
3427/3427 [==============================] - 4s 1ms/step - loss: 13.9255 - acc: 0.0849: 1s - loss: 14.0464
Epoch 2/20
3427/3427 [==============================] - 1s 402us/step - loss: 13.7073 - acc: 0.1004
Epoch 3/20
3427/3427 [==============================] - 1s 387us/step - loss: 13.7048 - acc: 0.0992
Epoch 4/20
3427/3427 [==============================] - 1s 376us/step - loss: 13.7027 - acc: 0.10300s - loss: 13.7417 - a
Epoch 5/20
3427/3427 [==============================] - 1s 371us/step - loss: 13.7007 - acc: 0.1036
Epoch 6/20
3427/3427 [==============================] - 1s 404us/step - loss: 13.6989 - acc: 0.1059
Epoch 7/20
3427/3427 [==============================] - 2s 450us/step - loss: 13.6977 - acc: 0.1062
Epoch 8/20
3427/3427 [==============================] - 1s 415us/step - loss: 13.6973 - acc: 0.1065
Epoc

3427/3427 [==============================] - 1s 392us/step - loss: 7.1672 - acc: 0.1217
Epoch 17/20
3427/3427 [==============================] - 1s 376us/step - loss: 7.1663 - acc: 0.1103
Epoch 18/20
3427/3427 [==============================] - 1s 380us/step - loss: 7.1634 - acc: 0.1188
Epoch 19/20
3427/3427 [==============================] - 2s 503us/step - loss: 7.1653 - acc: 0.1135
Epoch 20/20
381/381 [==============================] - 2s 4ms/step
Epoch 1/20
3428/3428 [==============================] - 6s 2ms/step - loss: 10.2314 - acc: 0.0916
Epoch 2/20
3428/3428 [==============================] - 2s 670us/step - loss: 9.6856 - acc: 0.0960 1s - loss: 9
Epoch 3/20
3428/3428 [==============================] - 2s 500us/step - loss: 9.6213 - acc: 0.0960
Epoch 4/20
3428/3428 [==============================] - 2s 603us/step - loss: 9.5993 - acc: 0.0980
Epoch 5/20
3428/3428 [==============================] - 2s 521us/step - loss: 9.5846 - acc: 0.1050
Epoch 6/20
3428/3428 [================

In [113]:
model = baseline_model()
ll = model.fit(X, dummy_y, 32, 20, verbose=1)
print(ll)

Epoch 1/20
3808/3808 [==============================] - 3s 850us/step - loss: 13.4316 - acc: 0.0349
Epoch 2/20
3808/3808 [==============================] - 0s 65us/step - loss: 8.2649 - acc: 0.0972
Epoch 3/20
3808/3808 [==============================] - 0s 66us/step - loss: 4.5884 - acc: 0.0930
Epoch 4/20
3808/3808 [==============================] - 0s 67us/step - loss: 3.2847 - acc: 0.1045
Epoch 5/20
3808/3808 [==============================] - 0s 64us/step - loss: 3.1690 - acc: 0.1045
Epoch 6/20
3808/3808 [==============================] - 0s 67us/step - loss: 3.1275 - acc: 0.1129
Epoch 7/20
3808/3808 [==============================] - 0s 65us/step - loss: 3.1034 - acc: 0.1179
Epoch 8/20
3808/3808 [==============================] - 0s 64us/step - loss: 3.0954 - acc: 0.1124
Epoch 9/20
3808/3808 [==============================] - 0s 65us/step - loss: 3.0863 - acc: 0.1155
Epoch 10/20
3808/3808 [==============================] - 0s 67us/step - loss: 3.0775 - acc: 0.1182
Epoch 11/20
3808/

In [72]:
model = Sequential()
model.add(LSTM(256, input_shape=(n_prev, 1), return_sequences=True))
model.add(LSTM(128, input_shape=(n_prev, 1), return_sequences=True))
model.add(LSTM(64, input_shape=(n_prev, 1), return_sequences=False))
model.add(Dense(1))
model.add(Activation('linear'))
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer='rmsprop')
filepath="./Checkpoints/checkpoint_model_{epoch:02d}.hdf5"
model_save_callback = ModelCheckpoint(filepath, monitor='val_acc', 
                                      verbose=1, save_best_only=False, 
                                      mode='auto', period=5)

# network_input = X
# model = Sequential()
# model.add(LSTM(
#         512,
#         input_shape=(network_input.shape[1], network_input.shape[2]),
#         return_sequences=True
#     ))
# model.add(Dropout(0.3))
# model.add(LSTM(512, return_sequences=True))
# model.add(Dropout(0.3))
# model.add(LSTM(512))
# model.add(Dense(256))
# model.add(Dropout(0.3))
# model.add(Dense(n_vocab))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

#### Train your model.
It might take a while, I was waiting for 1 hour with just 5 epoch

In [73]:
from keras.utils import to_categorical

ll = model.fit(np.array(X), np.array(y), 32, 50, verbose=1)

Epoch 1/50
3802/3802 [==============================] - 12s 3ms/step - loss: 0.0540
Epoch 2/50
3802/3802 [==============================] - 7s 2ms/step - loss: 0.0437
Epoch 3/50
3802/3802 [==============================] - 6s 2ms/step - loss: 0.0425
Epoch 4/50
3802/3802 [==============================] - 6s 2ms/step - loss: 0.0416
Epoch 5/50
3802/3802 [==============================] - 7s 2ms/step - loss: 0.0408
Epoch 6/50
3802/3802 [==============================] - 7s 2ms/step - loss: 0.0407
Epoch 7/50
3802/3802 [==============================] - 8s 2ms/step - loss: 0.0404
Epoch 8/50
3802/3802 [==============================] - 9s 2ms/step - loss: 0.0398
Epoch 9/50
3802/3802 [==============================] - 7s 2ms/step - loss: 0.0398
Epoch 10/50
3802/3802 [==============================] - 7s 2ms/step - loss: 0.0396
Epoch 11/50
3802/3802 [==============================] - 8s 2ms/step - loss: 0.0395
Epoch 12/50
3802/3802 [==============================] - 7s 2ms/step - loss: 0.0394


#### Make a prediction

In [114]:
prediction = model.predict(np.array(X_test))
prediction = np.squeeze(prediction)
prediction = np.squeeze(scaler.inverse_transform(prediction.reshape(-1,1)))
prediction = [int(i) for i in prediction]


#### Save your result to new midi file

In [117]:
print(prediction)
mid = MidiFile()
track = MidiTrack()
t = 0
for note in prediction:
    # 147 means note_on
    # 67 is velosity
    note = np.asarray([147, note, 67])
    bytes = note.astype(int)
    msg = Message.from_bytes(bytes[0:3])
    t += 1
    msg.time = t
    track.append(msg)
mid.tracks.append(track)
mid.save('20_epoch_8_note_seq_CE.mid')

[37, 37, 39, 37, 37, 39, 37, 37, 38, 39, 37, 37, 37, 40, 40, 39, 40, 38, 42, 41, 37, 37, 37, 37, 37, 37, 38, 37, 37, 39, 37, 37, 39, 38, 37, 37, 37, 41, 40, 39, 41, 38, 42, 41, 37, 37, 37, 37, 37, 37, 39, 37, 37, 38, 37, 37, 38, 39, 37, 37, 37, 39, 40, 40, 40, 38, 41, 41, 37, 37, 37, 37, 37, 37, 39, 37, 37, 39, 38, 38, 38, 39, 37, 37, 37, 40, 40, 39, 40, 39, 41, 40, 37, 37, 37, 37, 37, 37, 38, 37, 37, 39, 37, 38, 39, 38, 37, 37, 37, 41, 40, 39, 40, 38, 41, 40, 37, 37, 37, 37, 37, 37, 38, 37, 37, 39, 37, 37, 39, 38, 37, 37, 37, 40, 40, 39, 40, 38, 42, 41, 37, 37, 37, 37, 37, 37, 38, 37, 37, 39, 37, 37, 39, 38, 37, 37, 37, 40, 40, 39, 41, 38, 42, 41, 37, 37, 37, 37, 37, 37, 39, 37, 37, 39, 37, 37, 38, 38, 37, 37, 37, 40, 40, 39, 40, 38, 42, 41, 37, 37, 37, 37, 37, 37, 38, 37, 37, 39, 37, 37, 39, 38, 37, 37, 37, 40, 40, 39, 40, 38, 42, 41, 37, 37, 37, 37, 37, 37, 39, 37, 37, 39, 37, 37, 38, 38, 37, 37, 37, 40, 40, 39, 40, 38, 42, 41, 37, 37, 37, 37, 37, 37, 39, 37, 37, 39, 37, 37, 38, 38,

#### Just listen to it. The result is surreal!